In [28]:
import pandas as pd
import re

In [29]:
ref = pd.read_csv('reference.csv')
ref

,Unnamed: 0,filename
0,0,2019-05-10 TS_Mrs D [W1921].pdf
1,1,2019-06-06 TS_Peter Buckland [W0694].pdf
2,2,2019-05-06 TS_Michelle Baker [W1825].pdf
3,3,2020-02-25 TS_Psychosocial Experts (London).pdf
4,4,2019-05-01 TS_Carolyn Challis [W0622].pdf
...,...,...
362,362,"2019-07-05 TS_William Wright [W2287], Rosemary..."
363,363,2019-07-10 TS_Ms Y [W2288].pdf
364,364,"2019-07-09 TS_Mr W [W2781], Christine Norval [..."
365,365,2019-07-10 TS_Pauline Reid [W2247].pdf


In [30]:
at = pd.read_csv('all_transcripts.csv').rename(columns={'index':'org_index'}).dropna()

In [31]:
at

,Unnamed: 0,org_index,text_clean
0,0,0,\n \n
1,1,0,SIR BRIAN LANGSTAFF:
2,2,0,Our next witness will give evidence \nunder...
3,3,0,SIR BRIAN LANGSTAFF:
4,4,0,"I understand, Mrs D, that for your \ncomfor..."
...,...,...,...
307386,307386,366,Thank you. \n
307387,307387,366,SIR BRIAN LANGSTAFF:
307388,307388,366,10 o'clock tomorrow.\n
307389,307389,366,MS FRASER BUTLIN:


In [42]:
from whoosh.fields import Schema, TEXT, ID
from whoosh import index
import os, os.path
from whoosh import index
from whoosh import qparser
from whoosh.qparser import QueryParser

schema = Schema(org_idx=ID(stored=True), path=ID(stored=True), content=TEXT(stored = True))

In [43]:
if not os.path.exists("test_index_dir"):
    os.mkdir("test_index_dir")

In [44]:
ix = index.create_in("test_index_dir", schema)
writer = ix.writer()

for i in range(len(at)):
    writer.add_document(org_idx = str(at.org_index.iloc[i]), content=str(at.text_clean.iloc[i]), path=str(i))

writer.commit()

In [52]:
def index_search(dirname, search_fields, search_query):
    ix = index.open_dir(dirname)
    schema = ix.schema
    
    og = qparser.OrGroup.factory(0.9)
    mp = qparser.MultifieldParser(search_fields, schema, group = og)

    
    q = mp.parse(search_query)
    
    
    with ix.searcher() as s:
        results = s.search(q, terms=True)
        print("Search Results: ")
        
        
        return results[0:20]

In [53]:
results_dict = index_search("test_index_dir", ['content'], u"compensation AND Macfarlane")

Search Results: 


In [54]:
results_dict[0:10]

ReaderClosed: 

In [4]:
search = 'compensation'

In [6]:
loc_input = at.text_clean.str.contains(f'\s{search}\s')
search_trans = at.loc[loc_input]

In [15]:
search_trans[0:10]

,Unnamed: 0,org_index,text_clean
2487,2487,5,"No, I wasn't aware of it and I was just rerea..."
6348,6348,12,"You've described the process as: \n""... incre..."
10760,10760,22,"Yes, I just set some thoughts out. If you'll..."
11188,11188,24,The hepatitis C we only became aware of becau...
11767,11767,25,"I do, because my stage 1 is very close to wha..."
15480,15480,32,I want to just ask you a little about the lit...
15538,15538,32,"Okay. One of the things I do need to say, it..."
17747,17747,37,Means testing. What has my wife's income got...
21041,21041,45,"Okay. All right. Well, if you can't assist ..."
21049,21049,45,We can take that document down. Before I sho...


In [16]:
type(search_trans.to_csv().encode('utf-8'))

bytes

In [22]:
#with open('test.txt', 'w') as f: [f.write(f'\n--Result #{i+1}--\n{doc}') for i, doc in enumerate(search_trans.text_clean.to_list())]

type(''.join([f'\n--Result #{i+1}--\n{doc}' for i, doc in enumerate(search_trans.text_clean.to_list())]).encode('utf-8'))

bytes

In [30]:
search = 'compensation AND litigation'
if 'AND' in search:
    s_list = search.split('AND')
    search = ' '.join([f'(?=.*{s.strip()})' for s in s_list])

search

'(?=.*compensation) (?=.*litigation)'

In [31]:
search_trans = at.loc[(at.text_clean.str.contains(f'\s{search}\s'))]

In [32]:
search_trans

,Unnamed: 0,org_index,text_clean
